In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
import requests

from datetime import timedelta, date

import itertools

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
dates=[] #pasirinktų datų sąrašas nuskaitomiems duomenims

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2018, 1, 1) 
end_dt = date(2018, 8, 15)


for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))

dates=[d for d in dates if d not in ['2018-02-19', '2018-05-21']]
#Policijos departamento informacija. 
#lentele lentelej

In [3]:
dfs2 = [] #Lentelių sąrašas paros duomenų lentelei apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus sukurti
#dfs2 = [] Lentelių sąrašas paros gaisrų aprašymų lentelei sukurti

for dateurl in dates:
    url = 'http://www.vpgt.lt/vpgt/m/m_events_summary/wfiles/e_'+dateurl+'.htm'
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    
    data0=[] #duomenų sąrašas
                 
            
    for tr in rows:
        cols = tr.find_all('td')
        for td in cols:
            if td.attrs["style"] == "border:none":
                td.decompose()
            else:
                text1 = ""
                for text0 in td.find_all("p"):
                    text1 += text0.text + " "
            data0.append(text1.rstrip())

                
    #Paros duomenys apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
    data0 = [w.replace('\r\n ', '') for w in data0]
    data0 = [w.replace('\n', ' ') for w in data0]
    
    #Paros gaisrų aprašymų duomenys
    data2 = data0[data0.index('2. Gaisrų aprašymas')+1:data0.index('3. Gelbėjimo darbų aprašymas')]
    df02 = pd.DataFrame(np.array(data2).reshape(int(len(data2)/6),6))
    dfs2.append(df02)
    ############

    
############
#Paros gaisrų aprašymų lentelė
df2 = pd.concat(dfs2, ignore_index=True)
############

In [4]:
df2.head()

,0,1,2,3,4,5
0,1 2018-01-01 06:29,2018-01-01 04:24 04:33 04:42 04:42 05:29,"Kalvarijos sav., Kalvarijos sen., Kušliškių k....","Atvykus, degė automobilio salonas, žmonių vidu...","AC – 2, 1-B","VPGT-2, SPT-1, APGV tarnybos pavaldžių komand..."
1,2 2018-01-01 10:07,2018-01-01 07:30 07:46 08:03 08:24 09:22,"Vilniaus r., Mickūnų sen., Vališkių k., Darbin...","Atvykus, atvira liepsna degė gyvenamo namo sto...","AC – 3, 2-B, 1 (2) KOAA grandis","VPGT-6, SPT-4, Pamainos vadas"
2,2 2018-01-01 12:05,2018-01-01 09:50 09:56 10:07 10:08 11:05,"Kaišiadorių r., Kaišiadorių apylinkės sen., Gu...","Atvykus, namas buvo uždūmintas. Namas mūrinis,...","AC – 3, 1-B, 1 (3) KOAA grandis","VPGT-8, SPT-2, Pamainos vadas"
3,5 2018-01-01 14:51,2018-01-01 11:14 11:24 11:59 12:38 13:53,"Trakų r., Lentvario sen., Selioviškių k., Grig...","Atvykus, atvira liepsna degė medinis, 1 aukšto...","AC – 3, 3-B, 1 (2) KOAA grandis","VPGT-8, Pamainos vadas"
4,15 2018-01-01 16:48,2018-01-01 14:46 14:54 14:56 15:09 15:34,"Kauno m., Kražantės g. 18 Transporto priemonė","Atvykus, degė sunkvežimio MAN akumuliatoriaus ...","AC - 3, ŠA, 1-B","VPGT-14 Vyresnysis specialistas, Pamainos vadas"


In [5]:
df2.to_csv('2018.csv', index=False, header=True, encoding="cp1257")